In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functions_NLM import estimate_nested_logit, simulate_choice
from functions_geom_DIB import geom_DIB_on_alternatives

In [4]:
# Load data
data = pd.read_csv('../data/airline.dat', sep='\t')
# display every column of the dataframe
pd.set_option('display.max_columns', None)
data

,SubjectId,OriginGMT,DestinationGMT,Direction,q02_TripPurpose,q03_WhoPays,q11_DepartureOrArrivalIsImportant,q12_IdealDepTime,q13_IdealArrTime,q14_PartySize,q15_Age,q16_Income,Cont_Income,q17_Gender,q19_Occupation,q20_Education,AirlineFirstFlight_1,AirlineFirstFlight_2,AirlineFirstFlight_3,AirlineSecondFlight_1,AirlineSecondFlight_2,AirlineSecondFlight_3,AirplaneFirstFlight_1,AirplaneFirstFlight_2,AirplaneFirstFlight_3,AirplaneSecondFlight_1,AirplaneSecondFlight_2,AirplaneSecondFlight_3,DepartureTimeHours_1,DepartureTimeHours_2,DepartureTimeHours_3,DepartureTimeMins_1,DepartureTimeMins_2,DepartureTimeMins_3,ArrivalTimeHours_1,ArrivalTimeHours_2,ArrivalTimeHours_3,ArrivalTimeMins_1,ArrivalTimeMins_2,ArrivalTimeMins_3,FlyingTimeHours_1,FlyingTimeHours_2,FlyingTimeHours_3,TripTimeHours_1,TripTimeHours_2,TripTimeHours_3,Legroom_1,Legroom_2,Legroom_3,Fare_1,Fare_2,Fare_3,BestAlternative_1,BestAlternative_2,BestAlternative_3,TripPurpose
0,1,300,360,0,2,1,2,-1,480,2,3.5,9,92.5,1,2,6,5,6,3,-1,-1,2,2,3,7,-1,8,7,7.0,9.0,9.0,420,540,540,10.6167,14.1167,13.6167,637,847,817,4.616670,5.11667,5.11667,4.616670,6.11667,5.61667,2,3,4,835,835,730,0,0,1,2
1,2,480,480,1,2,1,2,-1,540,1,3.5,10,112.5,1,1,6,3,4,5,-1,-1,1,8,8,2,-1,8,1,7.5,15.0,12.0,450,900,720,10.0333,18.5333,16.5333,602,1112,992,2.533330,3.03333,3.03333,2.533330,3.53333,4.53333,2,2,3,315,315,350,0,1,0,2
2,3,480,480,1,2,1,1,-1,-1,2,3.5,6,60.0,1,2,6,5,9,2,-1,-1,6,8,5,5,-1,6,6,7.5,9.0,12.0,450,540,720,10.0333,13.5333,17.0333,602,812,1022,2.533330,3.03333,3.03333,2.533330,4.53333,5.03333,3,1,4,195,160,170,0,1,0,2
3,4,480,480,1,2,3,1,630,-1,2,6.0,12,162.5,2,2,6,4,3,7,-1,-1,6,7,4,6,-1,6,8,18.0,18.0,12.0,1080,1080,720,18.9667,21.4667,13.9667,1138,1288,838,0.966667,1.46667,1.46667,0.966667,3.46667,1.96667,3,3,1,135,125,140,0,0,1,2
4,5,300,480,0,2,1,0,-1,-1,3,5.0,10,112.5,2,5,7,7,1,4,-1,-1,3,3,2,2,-1,3,6,10.5,15.0,15.0,630,900,900,11.8000,18.3000,17.8000,708,1098,1068,4.300000,4.80000,4.80000,4.300000,6.30000,5.80000,3,2,3,270,300,300,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3604,3609,480,300,2,3,1,2,-1,660,3,4.0,3,30.0,2,5,2,7,3,3,-1,-1,9,5,7,5,-1,2,3,12.0,11.0,9.0,720,660,540,19.2667,19.2667,18.2667,1156,1156,1096,4.266670,4.76667,4.76667,4.266670,5.26667,6.26667,3,3,4,650,480,650,0,1,0,3
3605,3610,480,300,2,2,1,2,-1,480,1,3.0,5,50.0,1,2,7,1,2,7,-1,-1,10,7,8,8,-1,6,4,12.0,10.0,7.0,720,600,420,20.3333,19.8333,16.3333,1220,1190,980,5.333330,5.83333,5.83333,5.333330,6.83333,6.33333,2,3,4,760,690,590,0,0,1,2
3606,3611,300,360,0,1,1,2,-1,870,1,4.0,8,80.0,1,7,7,10,1,5,-1,-1,3,4,4,3,-1,3,6,9.0,16.5,12.0,540,990,720,10.3000,18.8000,14.3000,618,1128,858,2.300000,2.80000,2.80000,2.300000,3.30000,3.30000,2,1,4,290,290,300,0,0,1,1
3607,3612,300,480,0,2,1,2,-1,1320,1,3.0,10,112.5,1,1,7,7,5,4,-1,-1,6,5,1,6,-1,5,5,10.0,11.0,11.0,600,660,660,13.1500,16.6500,15.6500,789,999,939,6.150000,6.65000,6.65000,6.150000,8.65000,7.65000,3,4,3,480,360,420,1,0,0,2


In [5]:
# Find the column index with the maximum value (1)
choice_column_index = data[['BestAlternative_1', 'BestAlternative_2', 'BestAlternative_3']].idxmax(axis=1)

# Add 1 to the index to get the choice number
data['Choice'] = choice_column_index.str[-1].astype(int)

data_used = data[['SubjectId', 'TripTimeHours_1', 'TripTimeHours_2', 'TripTimeHours_3',
                   'Fare_1', 'Fare_2', 'Fare_3', 'Choice']]
data_used

,SubjectId,TripTimeHours_1,TripTimeHours_2,TripTimeHours_3,Fare_1,Fare_2,Fare_3,Choice
0,1,4.616670,6.11667,5.61667,835,835,730,3
1,2,2.533330,3.53333,4.53333,315,315,350,2
2,3,2.533330,4.53333,5.03333,195,160,170,2
3,4,0.966667,3.46667,1.96667,135,125,140,3
4,5,4.300000,6.30000,5.80000,270,300,300,1
...,...,...,...,...,...,...,...,...
3604,3609,4.266670,5.26667,6.26667,650,480,650,2
3605,3610,5.333330,6.83333,6.33333,760,690,590,3
3606,3611,2.300000,3.30000,3.30000,290,290,300,3
3607,3612,6.150000,8.65000,7.65000,480,360,420,1


In [13]:
# beta_names = ["ASC_2", "ASC_3", "BETA_Time", "BETA_Cost", "lambda_13",
#                "lambda_32", "base_alpha_3_WITH_1"]

# DEFINE MODEL STRUCTURE AND LIKELIHOOD FUNCTION
def log_likelihood_optima_CNLM(beta, data):
    alpha_3_WITH_1 = np.exp(beta[6]) / (1 + np.exp(beta[6]))
    alpha_3_WITH_2 = 1 - alpha_3_WITH_1

    # define utility functions
    data['U_1'] = beta[2] * data['TripTimeHours_1'] + beta[3] * data['Fare_1']
    data['U_2'] = beta[0] + beta[2] * data['TripTimeHours_2'] + beta[3] * data['Fare_2']
    data['U_3'] = beta[1] + beta[2] * data['TripTimeHours_3'] + beta[3] * data['Fare_3']
 
    # combined utility terms inside nests
    data['U_13'] = (alpha_3_WITH_1 * np.exp(data['U_3'])) ** (1 / beta[4]) + \
                          (1 * np.exp(data['U_1'])) ** (1 / beta[4])
    data['U_32'] = (alpha_3_WITH_2 * np.exp(data['U_3'])) ** (1 / beta[5]) + \
                         (1 * np.exp(data['U_2'])) ** (1 / beta[5])

    # Nest probabilities
    data['P_nest_13'] = data['U_13'] ** beta[4] / \
                                (data['U_13'] ** beta[4] + data['U_32'] ** beta[5])
    data['P_nest_32'] = 1 - data['P_nest_13']

    # Within nest probabilities for any nests with > 1 alt
    data['P_3_in_13'] = (alpha_3_WITH_1 * np.exp(data['U_3'])) ** (1 / beta[4]) / \
                                    data['U_13'] 
    data['P_1_in_13'] = 1 - data['P_3_in_13']
    data['P_3_in_32'] = (alpha_3_WITH_2 * np.exp(data['U_3'])) ** (1 / beta[5]) / \
                                data['U_32']
    data['P_2_in_32'] = 1 - data['P_3_in_32']

    # Full probabilities
    data['P_1'] = data['P_nest_13'] * data['P_1_in_13']
    data['P_2'] = data['P_nest_32'] * data['P_2_in_32']
    data['P_3'] = data['P_nest_13'] * data['P_3_in_13'] + data['P_nest_32'] * data['P_3_in_32']

    # calculate probability for chosen alternative for each row in the data table
    data['P'] = ((data['Choice'] == 1) * data['P_1'] +
                 (data['Choice'] == 2) * data['P_2'] +
                 (data['Choice'] == 3) * data['P_3'])

    # take product across choices for the same person (likelihood)
    # then take the log for log-likelihood
    LL = data.groupby('SubjectId')['P'].prod().apply(np.log).sum()

    return -LL  # We minimize negative log-likelihood

In [15]:
# Define model parameters
beta = np.array([0, 0, 0, 0, 1, 1, 1])
beta_names = ["ASC_2", "ASC_3", "BETA_Time", "BETA_Cost", "lambda_13",
              "lambda_32", "base_alpha_3_WITH_1"]
# Estimate parameters
result, se, t_stat, p_value, aic, bic  = estimate_nested_logit(data_used, beta, beta_names, log_likelihood_optima_CNLM)

/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_1704/726278811.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['U_1'] = beta[2] * data['TripTimeHours_1'] + beta[3] * data['Fare_1']
/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_1704/726278811.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['U_2'] = beta[0] + beta[2] * data['TripTimeHours_2'] + beta[3] * data['Fare_2']
/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_1704/726278811.py:12: SettingWithCopyWa

Optimization Results:
             Parameter  Estimate  Robust Asymptotic SE  t-statistic  \
0                ASC_2 -1.011324              0.085263   -11.861297   
1                ASC_3 -1.154200              0.078956   -14.618182   
2            BETA_Time -0.257938              0.043028    -5.994618   
3            BETA_Cost -0.015801              0.000679   -23.276125   
4            lambda_13  0.126798              0.156339     0.811049   
5            lambda_32  0.530093              0.035338    15.000742   
6  base_alpha_3_WITH_1 -4.790131              1.547354    -3.095691   

        p-value  
0  0.000000e+00  
1  0.000000e+00  
2  2.240549e-09  
3  0.000000e+00  
4  4.173909e-01  
5  0.000000e+00  
6  1.978638e-03  
AIC: 4809.062073575499
BIC: 4852.400375607998
Log-likelihood: -2397.5310367877496


In [6]:
df_input = data[['TripTimeHours_1', 'TripTimeHours_2', 'TripTimeHours_3',
                   'Fare_1', 'Fare_2', 'Fare_3']]
# Calculate frequencies and probabilities
vector_counts = df_input.value_counts().reset_index(name='Frequency')
vector_counts['Probability'] = vector_counts['Frequency'] / len(df_input)

# Creating a tuple of attributes to facilitate mapping
vector_counts['tuple'] = vector_counts[['TripTimeHours_1', 'TripTimeHours_2', 'TripTimeHours_3',
                                        'Fare_1', 'Fare_2', 'Fare_3']].apply(tuple, axis=1)
probability_map = vector_counts.set_index('tuple')['Probability'].to_dict()

# Assign the probability to each row based on its tuple of attributes
df_input['Probability'] = df_input.apply(lambda row: probability_map[tuple(row)], axis=1)


/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_13239/1916923433.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['Probability'] = df_input.apply(lambda row: probability_map[tuple(row)], axis=1)


In [7]:
# Computation of p(x,y)
p_x = df_input['Probability'].values

p_y_given_x = data_used[['P_1', 'P_2', 'P_3']].values
p_xy = p_x[:, np.newaxis] * p_y_given_x

# Normalize p_xy 
p_xy /= p_xy.sum()

# Define epsilon value
epsilon = 1e-20

# Add epsilon to elements equal to 0
p_xy[p_xy == 0] += epsilon

In [8]:
def simulate_choice_optima(row):
    """
    Function to simulate a choice based on cumulative probabilities.

    Parameters:
    - row: pandas Series or DataFrame row containing probabilities for each choice

    Returns:
    - The simulated choice (an integer between 1 and the number of choices)
    """
    # Generate a random number between 0 and 1
    random_number = np.random.rand()
    
    # Calculate cumulative probabilities
    probabilities = row[['P_1', 'P_2', 'P_3']]
    cumulative_probs = probabilities.cumsum()
    
    # Determine simulated choice based on random number
    for i, cumulative_prob in enumerate(cumulative_probs):
        if random_number <= cumulative_prob:
            return i+1

In [9]:
np.random.seed(123)
data_used['simulated_choice'] = data_used.apply(simulate_choice_optima, axis=1)
data_used

/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_13239/2856924636.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_used['simulated_choice'] = data_used.apply(simulate_choice_optima, axis=1)


,SubjectId,TripTimeHours_1,TripTimeHours_2,TripTimeHours_3,Fare_1,Fare_2,Fare_3,Choice,U_1,U_2,U_3,U_13,U_32,P_nest_13,P_nest_32,P_3_in_13,P_1_in_13,P_3_in_32,P_2_in_32,P_1,P_2,P_3,P,simulated_choice
0,1,4.616670,6.11667,5.61667,835,835,730,3,-16.223312,-18.753455,-16.635709,1.111072e-16,5.468693e-13,0.616681,0.383319,0.321965,0.678035,0.943807,0.056193,0.418132,0.021540,0.560329,0.560329,3
1,2,2.533330,3.53333,4.53333,315,315,350,2,-6.338463,-8.730746,-9.533236,7.142555e-08,5.796120e-07,0.776644,0.223356,0.003544,0.996456,0.116729,0.883271,0.773892,0.197284,0.028825,0.197284,1
2,3,2.533330,4.53333,5.03333,195,160,170,2,-4.189903,-6.231244,-6.448256,8.961708e-06,4.367572e-05,0.693596,0.306404,0.026683,0.973317,0.258760,0.741240,0.675089,0.227119,0.097792,0.227119,1
3,4,0.966667,3.46667,1.96667,135,125,140,3,-2.683661,-5.310479,-5.065573,2.500189e-04,2.612027e-04,0.775194,0.224806,0.020614,0.979386,0.428969,0.571031,0.759215,0.128371,0.112414,0.112414,1
4,5,4.300000,6.30000,5.80000,270,300,300,1,-6.019862,-9.225006,-8.987251,1.457703e-07,3.928611e-07,0.858376,0.141624,0.005838,0.994162,0.426065,0.573935,0.853365,0.081283,0.065352,0.853365,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3604,3609,4.266670,5.26667,6.26667,650,480,650,2,-12.814446,-12.162935,-15.382555,4.166956e-14,1.726926e-09,0.152881,0.847119,0.013877,0.986123,0.002390,0.997610,0.150759,0.845095,0.004146,0.845095,2
3605,3610,5.333330,6.83333,6.33333,760,690,590,3,-15.078060,-16.354877,-14.326654,1.550268e-14,2.544007e-11,0.595030,0.404970,0.389103,0.610897,0.935391,0.064609,0.363502,0.026165,0.610333,0.610333,3
3606,3611,2.300000,3.30000,3.30000,290,290,300,3,-5.826513,-8.218795,-8.297947,2.305779e-07,1.722302e-06,0.753530,0.246470,0.017058,0.982942,0.304979,0.695021,0.740677,0.171302,0.088021,0.088021,1
3607,3612,6.150000,8.65000,7.65000,480,360,420,1,-10.289927,-10.947231,-11.645895,1.680677e-11,1.498024e-08,0.422487,0.577513,0.138162,0.861838,0.135694,0.864306,0.364115,0.499148,0.136737,0.364115,2


In [10]:
betas = np.linspace(0, 50, 51)
# Initialize an empty list to store the number of clusters
num_clusters_list = []

# Iterate over each beta value
for beta in betas:
    # Run iterative_algorithm to obtain q_t_given_x
    q_t_given_x, _, _ = geom_DIB_on_alternatives(p_xy, max_iter=5000, beta=beta, threshold=1e-4)
    
    # Calculate the number of clusters
    column_sum = np.sum(q_t_given_x, axis=0)
    num_clusters = np.count_nonzero(column_sum)
    
    # Append the number of clusters to the list
    num_clusters_list.append(num_clusters)

# Plot the number of clusters against beta values
plt.plot(betas, num_clusters_list)
plt.xlabel('Beta')
plt.ylabel('Number of Clusters')
plt.title('Number of Clusters vs. Beta')
plt.grid(True)
plt.show()

Iteration: 1 out of 5000
Objective function value: 1.0985997641372804
H(T) =  1.0985997641372804
I(T;Y) =  0.00039412985445785864
Iteration: 2 out of 5000
Objective function value: 0.7958394297761275
H(T) =  0.7958394297761275
I(T;Y) =  8.514959273986378e-05
Iteration: 3 out of 5000
Objective function value: 0.5999047342041968
H(T) =  0.5999047342041968
I(T;Y) =  5.453853460379854e-05
Iteration: 4 out of 5000
Objective function value: 0.4849298131698601
H(T) =  0.4849298131698601
I(T;Y) =  4.011567353678913e-05
Iteration: 5 out of 5000
Objective function value: 0.4093453736132401
H(T) =  0.4093453736132401
I(T;Y) =  3.172550685892617e-05
Iteration: 6 out of 5000
Objective function value: 0.3556297103567202
H(T) =  0.3556297103567202
I(T;Y) =  2.6237812516205494e-05
Iteration: 7 out of 5000
Objective function value: 0.31533241620882996
H(T) =  0.31533241620882996
I(T;Y) =  2.2368599980504555e-05
Iteration: 8 out of 5000
Objective function value: 0.2838855203136473
H(T) =  0.283885520313

KeyboardInterrupt: 

In [11]:
q_t_given_x, q_t, q_y_given_t = geom_DIB_on_alternatives(p_xy, beta=20, max_iter=5000, threshold=1e-4)

Iteration: 1 out of 5000
Objective function value: 1.0907171670481233
H(T) =  1.0985997641372804
I(T;Y) =  0.00039412985445785864
Iteration: 2 out of 5000
Objective function value: 0.20841284881238864
H(T) =  1.0623782937343622
I(T;Y) =  0.04269827224609868
Iteration: 3 out of 5000
Objective function value: -0.8018122943376795
H(T) =  1.0001966494557344
I(T;Y) =  0.0901004471896707
Iteration: 4 out of 5000
Objective function value: -1.12533516508205
H(T) =  0.9397716698882662
I(T;Y) =  0.1032553417485158
Iteration: 5 out of 5000
Objective function value: -1.1695795311554167
H(T) =  0.9076729719102663
I(T;Y) =  0.10386262515328415
Iteration: 6 out of 5000
Objective function value: -1.1901000405465543
H(T) =  0.8835479243042078
I(T;Y) =  0.10368239824253811
Iteration: 7 out of 5000
Objective function value: -1.2014038459614214
H(T) =  0.8635765931703392
I(T;Y) =  0.10324902195658803
Iteration: 8 out of 5000
Objective function value: -1.206825476791031
H(T) =  0.8478737186007717
I(T;Y) = 

In [12]:
import re

# Define a function to extract the number from the column name
def extract_number(column_name):
    match = re.search(r'\d+', column_name)  # Search for one or more digits in the column name
    if match:
        return int(match.group())  # Return the first occurrence of digits as an integer
    else:
        return None  # Return None if no digits are found

In [13]:
# Calculate the number of clusters
column_sum = np.sum(q_t_given_x, axis=0)
num_clusters = np.count_nonzero(column_sum)
print('Number of clusters :', num_clusters)

# Count the number of observations in each cluster
data_used['cluster'] = np.argmax(q_t_given_x, axis=1)
print('\n', data_used['cluster'].value_counts())

# number of each alternative 1, 2, 3, 4, 5 in each cluster 
cluster_counts = data_used.groupby(['cluster', 'Choice']).size().unstack(fill_value=0)
print('\n', cluster_counts)

data_used['max_proba'] = data_used[['P_1', 'P_2', 'P_3']].idxmax(axis=1).apply(extract_number)
cluster_counts2 = data_used.groupby(['cluster', 'max_proba']).size().unstack(fill_value=0)
print('\n', cluster_counts2)

cluster_counts3 = data_used.groupby(['cluster', 'simulated_choice']).size().unstack(fill_value=0)
print('\n', cluster_counts3)

Number of clusters : 3

 cluster
1    2558
2     621
0     430
Name: count, dtype: int64

 Choice      1    2    3
cluster                
0         185  217   28
1        2050  307  201
2         269   65  287

 max_proba     1    2    3
cluster                  
0           180  250    0
1          2558    0    0
2           353    0  268

 simulated_choice     1    2    3
cluster                         
0                  174  220   36
1                 2010  335  213
2                  303   44  274


/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_13239/687862934.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_used['cluster'] = np.argmax(q_t_given_x, axis=1)
/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_13239/687862934.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_used['max_proba'] = data_used[['P_1', 'P_2', 'P_3']].idxmax(axis=1).apply(extract_number)
